# Proyecto de Clasificación de IDs Válidos

Este notebook tiene como objetivo clasificar IDs en válidos o no válidos utilizando un modelo de Random Forest. Los datos se extraen de una base de datos Teradata y se procesan en lotes debido al gran tamaño del conjunto de datos.


## Configuración del Entorno

Primero, importamos las librerías necesarias para la ejecución del notebook:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import check_ids, get_connection, close_connection
from teradataml import execute_sql
import pandas as pd
import joblib
import math
from teradataml.dataframe.fastload import fastload

## Configuración Inicial

Definimos los parámetros y rutas necesarios para el procesamiento de datos:

- **`batch_size`**: Tamaño del lote para el procesamiento de datos.
- **`query_file`**: Ruta al archivo SQL que contiene la consulta para extraer los datos.
- **`columns`**: Lista de nombres de columnas esperadas en el DataFrame.


In [18]:
batch_size = 1000000
query_file = '../sql/queries/customerid_inactive_2m.sql'
columns = ['UNIDAD DE NEGOCIO', 'ZONA','REGION','CUSTOMER_ID', 'LLAVE','MIN FECHA','MAX FECHA','TRANSACTIONS','MONEY','FECHAS','VOLUME','QTY']

## Establecimiento de Conexión

Conectamos a la base de datos Teradata utilizando la función `get_connection()`, que se encarga de establecer la conexión y verificar que las credenciales estén correctas.


In [11]:
get_connection()

Conexión a Teradata Vantage establecida correctamente.


## Carga del Modelo y Preprocesador

Cargamos los modelos y preprocesadores necesarios para la clasificación de IDs. Utilizamos `joblib` para cargar el preprocesador y el modelo de Random Forest preentrenado desde archivos pickle (.pkl).


In [12]:
preprocessor = joblib.load('../models/preprocessor.pkl')
model = joblib.load('../models/random_forest_model.pkl')

## Extracción de Datos en Lotes

Procedemos a extraer y procesar los datos en lotes debido al gran tamaño del conjunto de datos. Calculamos el número total de registros y la cantidad de lotes necesarios, y luego ejecutamos la consulta SQL para cada lote.


In [19]:
all_data = pd.DataFrame()

with open(query_file, 'r') as file:
    query_template = file.read()

In [20]:
count_query = """
    select count(*) from col_mkt.info_per_customer C
    left join 
    col_mkt.customer_predictions P
    on C.UNIDAD_DE_NEGOCIO = P.UNIDAD_DE_NEGOCIO
    AND C.ZONA = P.ZONA
    AND C.REGION = P.REGION
    AND C.CUSTOMER_ID = P.CUSTOMER_ID
    WHERE P.CUSTOMER_ID IS NULL;
    """

total_records = pd.DataFrame(execute_sql(count_query))[0][0]
total_batches = math.ceil(total_records / batch_size)
print(total_batches)

22


## Procesamiento de Datos y Predicción

Para cada lote de datos:

1. **Conversión de Datos**: Convertimos las columnas a los formatos necesarios.
2. **Aplicación del Modelo**: Usamos el modelo para predecir la validez de los IDs.
3. **Carga de Resultados**: Insertamos los resultados en una tabla auxiliar en Teradata y luego en la tabla final.
4. **Manejo de Errores**: Capturamos y mostramos errores en caso de problemas durante la ejecución.

In [ ]:
for batch in range(total_batches):
    print(batch)
    start = batch * batch_size + 1
    end = start + batch_size - 1
    query = query_template.format(start=start, end=end)
    try:
        
        result = execute_sql(query)
        print('Consulta hecha correctamente.')
        batch_df = pd.DataFrame(result, columns=columns)
        batch_df['VOLUME'] = pd.to_numeric(batch_df['VOLUME'], errors='coerce')
        batch_df['QTY'] = pd.to_numeric(batch_df['QTY'], errors='coerce')
        batch_df['MONEY'] = pd.to_numeric(batch_df['MONEY'], errors='coerce')
        batch_df['MIN FECHA'] = pd.to_datetime(batch_df['MIN FECHA'], format='%Y-%m-%d')
        batch_df['MAX FECHA'] = pd.to_datetime(batch_df['MAX FECHA'], format='%Y-%m-%d')
        batch_df[['ID_LENGTH', 'ID_HAS_REPEATED_DIGITS', 'ID_IS_ASCENDING', 'ID_IS_DESCENDING', 'ID_HAS_REPETITIVE_PATTERN']] = batch_df[
            'CUSTOMER_ID'].apply(lambda x: pd.Series(check_ids(x)))
        
        input_processed = preprocessor.transform(batch_df)
        prediction = model.predict(input_processed)
        
        batch_df['IS_VALID'] = prediction
        batch_df = batch_df[['UNIDAD DE NEGOCIO','ZONA','REGION','CUSTOMER_ID','IS_VALID']]
        
        print('Insertando en Teradata...')
        
        fastload(df=batch_df, schema_name='COL_MKT', table_name='CUSTOMER_PREDICTIONS_AUX',if_exists='replace')
        
        print('Inserción auxiliar hecha')
        
        insert_query_predictions = """
            INSERT INTO COL_MKT.CUSTOMER_PREDICTIONS
            SELECT * FROM COL_MKT.CUSTOMER_PREDICTIONS_AUX;
            """
        
        
        execute_sql(insert_query_predictions)
        
        print(f'Inserción hecha de {batch_df.shape[0]}')
        
        all_data = pd.concat([all_data, batch_df], ignore_index=True)
        
    except Exception as e:
        print(f'Error al ejecutar la consulta para el lote {batch}: {e}')

0
Consulta hecha correctamente.
Insertando en Teradata...
Processed 108857 rows in batch 1.
Processed 108857 rows in batch 2.
Processed 108857 rows in batch 3.
Processed 108857 rows in batch 4.
Processed 108857 rows in batch 5.
Processed 108857 rows in batch 6.
Processed 108857 rows in batch 7.
Processed 217711 rows in batch 8.
Inserción auxiliar hecha
Inserción hecha de 979710
1
Consulta hecha correctamente.
Insertando en Teradata...
Processed 109591 rows in batch 1.
Processed 109591 rows in batch 2.
Processed 109591 rows in batch 3.
Processed 109591 rows in batch 4.
Processed 109591 rows in batch 5.
Processed 109591 rows in batch 6.
Processed 109591 rows in batch 7.
Processed 219179 rows in batch 8.
Inserción auxiliar hecha
Inserción hecha de 986316
2
Consulta hecha correctamente.
Insertando en Teradata...
Processed 107401 rows in batch 1.
Processed 107401 rows in batch 2.
Processed 107401 rows in batch 3.
Processed 107401 rows in batch 4.
Processed 107401 rows in batch 5.
Processed 

## Cierre de Conexión

Al finalizar el procesamiento de todos los lotes, cerramos la conexión a la base de datos utilizando la función `close_connection()`. Esto asegura que no queden conexiones abiertas y libera recursos.

In [9]:
close_connection()

Conexión cerrada correctamente.
